In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv("AI.csv",header=0)

D:\Programs\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (21,24) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
df.head()

,Unnamed: 0,AI_DATE,AC_MODEL,AC_SN,OPERATOR_CODE,EVENT_RECORD_IDENTIFIER,DIVERTED_LANDING,AIR_TURN_BACK,REJECTED_TAKE_OFF,GROUND_TURN_BACK,...,ATA_CAUSE,INTERRUPTION_REASON,CORRECTIVE_ACTION,AI_ID,EDIT_DATE,EDITED_BY,Unnamed: 23,ATA_CHAPTER,ATA_CHAPTER_CAUSE,AI_DATE_1
0,0,29-Mar-20,CRJ700,15299,EDV,NaN,0,N,N,Y,...,308102,ICE DET (X) FAIL - STATUS,DEFERRED - REPLACED ICE DETECTOR,590931,13-May-20,T12540,NaN,30,30,20
1,1,30-Mar-20,CRJ700,15125,EDV,NaN,0,N,N,Y,...,243101,TRU FAN FAIL - STATUS,DEFERRED - REPLACED TRU,590932,13-May-20,T12540,NaN,24,24,20
2,2,30-Mar-20,CRJ700,10176,EDV,NaN,0,N,N,Y,...,361315,ISOL FAIL - CAUTION,REPLACED ISOLATION VALVE,590933,13-May-20,T12540,NaN,36,36,20
3,3,3-Apr-20,CRJ700,15046,JZA,NaN,0,N,N,Y,...,302201,Incident: RTRN L COWL ANTI ICE CAUTION MSG. L ...,(Resolved On W/O: 684248 Taskcard: NR-00001 ) ...,591076,27-May-20,T12540,NaN,30,30,20
4,4,8-Apr-20,CRJ700,15044,JZA,NaN,0,N,N,Y,...,213108,Incident: RTRN OVBD COOL CAUTION MESSAGE DISPL...,(Resolved On W/O: 684350 Taskcard: NR-00001 ) ...,591077,27-May-20,T12540,NaN,21,21,20


In [3]:
len(df)

51781

In [5]:
df['features'] = df['INTERRUPTION_REASON'].str.cat(df['CORRECTIVE_ACTION'], sep='  ',na_rep=' ')

In [67]:
import nltk
def identify_tokens(df):
    comment = df['features']
    tokens = nltk.word_tokenize(comment)
    # taken only words (not punctuation)
    token_words = [w for w in tokens if w.isalpha()]
    return token_words

df['features_1'] = df.apply(identify_tokens, axis=1)


In [ ]:
from nltk.corpus import stopwords
stops = set(stopwords.words("english"))                  

def remove_stops(df):
    my_list = df['features_1']
    meaningful_words = [w for w in my_list if not w in stops]
    return (meaningful_words)

df['meaningful'] = df.apply(remove_stops, axis=1)

In [ ]:


def rejoin_words(df):
    my_list = df['meaningful']
    joined_words = ( " ".join(my_list))
    return joined_words

df['processed'] = df.apply(rejoin_words, axis=1)


In [88]:
df['processed']

0        ICE DET X FAIL STATUS DEFERRED REPLACED ICE DE...
1                TRU FAN FAIL STATUS DEFERRED REPLACED TRU
2               ISOL FAIL CAUTION REPLACED ISOLATION VALVE
3        Incident RTRN L COWL ANTI ICE CAUTION MSG L CO...
4        Incident RTRN OVBD COOL CAUTION MESSAGE DISPLA...
                               ...                        
51776    Pilot Base Heat caution message Removed replac...
51777    Attempted retract gear departure gear disagree...
51778    Tat probe heat R Aoa heat r static heat R pito...
51779    After departure NOSE DOOR OPEN GEAR DISAGREE N...
51780    APU OIL TEMP APU OIL PRESS CAUTION MESSAGE IN ...
Name: processed, Length: 51781, dtype: object

In [107]:
from sklearn.model_selection import train_test_split
X = df['processed']
y = df['ATA_CHAPTER_CAUSE']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])
text_clf.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [108]:
predictions = text_clf.predict(X_test)

from sklearn import metrics
metrics.confusion_matrix(y_test,predictions)

array([[221,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ..., 161,   0,   0],
       [  0,   0,   0, ...,   2, 530,   0],
       [  0,   0,   0, ...,   2,   0,   0]], dtype=int64)

In [109]:
metrics.accuracy_score(y_test,predictions)

0.8679775280898876

In [79]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
vectorizer =  CountVectorizer()
counts = vectorizer.fit_transform(X_train.values)
classifier = MultinomialNB()
targets = y_train.values
classifier.fit(counts, targets)

MultinomialNB()

In [80]:
example_counts = vectorizer.transform(X_test)
predictions = classifier.predict(example_counts)
metrics.confusion_matrix(y_test,predictions)

array([[221,   0,   1, ...,   0,   0,   0],
       [  0,   0,   2, ...,   0,   0,   0],
       [  0,   0, 265, ...,   0,   0,   0],
       ...,
       [  0,   0,   1, ...,   0,  19,   0],
       [  0,   0,   1, ...,   0,  69,   0],
       [  0,   0,   0, ...,   0,   1,   0]], dtype=int64)

In [81]:
metrics.accuracy_score(y_test,predictions)

0.29728464419475653

In [73]:
#X_train

In [57]:
#paragraph = "";
#for y,column in X_train.items():
#   print(column)
#   paragraph+=column

In [68]:
#for features, data in df['features']:
	#print("col_name:",features, "\ndata:",data)
#paragraph = "";
#for column in X_train[['features']]:
#   print(df['features'])
#   paragraph+=df['features'].values
#comment = df['features'].values
#comment
#array 

In [82]:
from sklearn.linear_model import LogisticRegression
lr_model = Pipeline([('tfidf',TfidfVectorizer()),('lr',LogisticRegression())])
lr_model.fit(X_train,y_train)

D:\Programs\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('lr', LogisticRegression())])

In [83]:
predictions = lr_model.predict(X_test)
metrics.confusion_matrix(y_test,predictions)

array([[221,   0,   1, ...,   0,   0,   0],
       [  0,   0,   1, ...,   0,   0,   0],
       [  0,   0, 299, ...,   0,   0,   0],
       ...,
       [  0,   0,   1, ...,  17,   7,   0],
       [  0,   0,   0, ...,   0,  71,   0],
       [  0,   0,   0, ...,   0,   1,   0]], dtype=int64)

In [84]:
metrics.accuracy_score(y_test,predictions)

0.47477762172284643

In [85]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=141)
sv_model = Pipeline([('tfidf',TfidfVectorizer()),('lr',knn)])
sv_model.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('lr', KNeighborsClassifier(n_neighbors=141))])

In [86]:
predictions = sv_model.predict(X_test)
metrics.confusion_matrix(y_test,predictions)

array([[221,   0,   0, ...,   0,   0,   0],
       [  0,   0,   1, ...,   0,   0,   0],
       [  2,   0, 198, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   3,  13,   0],
       [  0,   0,   0, ...,   0,  82,   0],
       [  0,   0,   0, ...,   0,   1,   0]], dtype=int64)

In [87]:
metrics.accuracy_score(y_test,predictions)

0.41426732209737827